In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

import os
from dotenv import load_dotenv
import boto3
import io
import pymysql
import json

DB Connection

In [2]:
from sshtunnel import SSHTunnelForwarder

class SSHMySQLConnector:
    def __init__(self):
        self.ssh_host = None
        self.ssh_username = None
        self.ssh_password = None
        self.db_username = None
        self.db_password = None
        self.db_name = None
        self.tunnel = None
        self.connection = None

    def load_config_from_json(self, json_path):
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                config = json.load(f)
                self.ssh_host = config['ssh_host']
                self.ssh_username = config['ssh_username']
                self.ssh_password = config['ssh_password']
                self.db_username = config['db_username']
                self.db_password = config['db_password']
                self.db_name = config['db_name']
        except Exception as e:
            print("❌ 설정 JSON 로딩 실패:", e)


    def connect(self):
        try:
            self.tunnel = SSHTunnelForwarder(
                (self.ssh_host, 22),
                ssh_username=self.ssh_username,
                ssh_password=self.ssh_password,
                remote_bind_address=('127.0.0.1', 3306),
            )
            self.tunnel.start()
            
            self.connection = pymysql.connect(
                host='127.0.0.1',
                port=self.tunnel.local_bind_port,
                user=self.db_username,
                password=self.db_password,
                db=self.db_name,
                cursorclass=pymysql.cursors.DictCursor  # 이 줄 추가
            )
            print("✅ DB 접속 성공")
        except Exception as e:
            print("❌ SSH 또는 DB 연결 실패:", e)

    def insert_query_with_lookup(self, table_name, data_list):
        try:
            with self.connection.cursor() as cursor:
                for data in data_list:
                    # 1. op_member에서 uid, user_id 조회
                    cursor.execute("""
                        SELECT uid, user_id FROM op_member
                        WHERE add1 = %s
                        LIMIT 1
                    """, (data['acnt_nm'],))
                    result = cursor.fetchone()
                    if result:
                        data['member_uid'] = result['uid']
                        data['user_id'] = result['user_id']
                        # 향후에 ig_user_id가 추가가 된다면, 해당 부분도 확인해서 추가할 수 있게
                        # data['ig_user_id'] = result['ig_user_id']
                    else:
                        data['member_uid'] = 0
                        data['user_id'] = 'None'
                        # data['ig_user_id'] = 'None'

                    # 2. INSERT 쿼리 구성 및 실행
                    columns = ', '.join(data.keys())
                    placeholders = ', '.join([f"%({k})s" for k in data.keys()])
                    insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                    cursor.execute(insert_sql, data)

                    print(f"✅ inserted acnt_id: {data.get('acnt_id', 'N/A')}")

            self.connection.commit()
        except Exception as e:
            self.connection.rollback()
            print("❌ INSERT 실패:", e)

    def close(self):
        if self.connection:
            self.connection.close()
        if self.tunnel:
            self.tunnel.stop()

In [4]:
ssh = SSHMySQLConnector()
ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
ssh.connect()

# data_list = [
#     {
#         'activity_score': 0.18618008163528707,
#         'trend_score': 1.2384506112322682,
#         'follower_total_engagement': 0.9006925508419993,
#         'follower_retention_rate': 4.84593837535013,
#         'avg_post_efficiency': 1.4776093002849746,
#         'acnt_id': '17841400070132367',
#         'acnt_nm': 'gnuoyeatt',
#         'influencer_scale_type': 'nano'
#     }
# ]

# ssh.insert_query_with_lookup('op_mem_seller_score', data_list)


✅ DB 접속 성공


Data Loading

In [ ]:
def load_s3_instagram_data():

    load_dotenv()
    aws_access_key = os.getenv("aws_accessKey")
    aws_secret_key = os.getenv("aws_secretKey")
    region_name='ap-northeast-2'

    # S3 클라이언트 생성
    client = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

    today = datetime.now()
    year, week, _ = today.isocalendar()

    bucket_name = 'flexmatch-data'

    table_list = ['RECENT_USER_INFO_MTR', 'TIME_SERIES_PROFILE_INFO', 'BY_USER_ID_MEDIA_DTL_INFO', 'BY_DATE_MEDIA_AGG_INFO']
    df_by_table_list = {}

    for table_name in table_list:
        prefix = f'instagram-data/tables/{table_name}/year={year}/week=25/' # 이번주가 없어서 week를 따로 x
        response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

        if 'Contents' in response:
            files = [content['Key'] for content in response['Contents'] if content['Key'].endswith('merged_data.parquet')]

            if not files:
                print(f"[Info] No merged_data.parquet found for table: {table_name}")
                continue

            # 각 파일 순회
            for file_key in files:
                try:
                    obj = client.get_object(Bucket=bucket_name, Key=file_key)
                    df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
                    df_by_table_list[table_name] = df
                    print(f"[Success] Loaded {file_key} for table {table_name}")
                except Exception as e:
                    print(f"[Error] Failed to read {file_key} for table {table_name}: {e}")
        else:
            print(f"[Info] No files found under prefix: {prefix}")
        
    return df_by_table_list

df_by_table_list = load_s3_instagram_data()

# recent_user_info_mtr_2 = df_by_table_list['RECENT_USER_INFO_MTR']
# time_series_profile_info_2 = df_by_table_list['TIME_SERIES_PROFILE_INFO']
# by_user_id_media_dtl_info_2 = df_by_table_list['BY_USER_ID_MEDIA_DTL_INFO']
# by_date_media_agg_info_2 = df_by_table_list['BY_DATE_MEDIA_AGG_INFO']

recent_user_info_mtr = df_by_table_list['RECENT_USER_INFO_MTR']
time_series_profile_info = df_by_table_list['TIME_SERIES_PROFILE_INFO']
by_user_id_media_dtl_info = df_by_table_list['BY_USER_ID_MEDIA_DTL_INFO']
by_date_media_agg_info = df_by_table_list['BY_DATE_MEDIA_AGG_INFO']



[Success] Loaded instagram-data/tables/RECENT_USER_INFO_MTR/year=2025/week=25/merged_data.parquet for table RECENT_USER_INFO_MTR
[Success] Loaded instagram-data/tables/TIME_SERIES_PROFILE_INFO/year=2025/week=25/merged_data.parquet for table TIME_SERIES_PROFILE_INFO
[Success] Loaded instagram-data/tables/BY_USER_ID_MEDIA_DTL_INFO/year=2025/week=25/merged_data.parquet for table BY_USER_ID_MEDIA_DTL_INFO
[Success] Loaded instagram-data/tables/BY_DATE_MEDIA_AGG_INFO/year=2025/week=25/merged_data.parquet for table BY_DATE_MEDIA_AGG_INFO


In [107]:
def load_weekly_instagram_data(bucket_name, table_list, weeks_back=2, target_filename='merged_data.parquet'):
    # 환경 변수 로딩
    load_dotenv()
    aws_access_key = os.getenv("aws_accessKey")
    aws_secret_key = os.getenv("aws_secretKey")

    client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name='ap-northeast-2'
    )

    # 주차 리스트 생성 
    today = datetime.now()
    weekday = today.weekday()
    # week_year_pairs = [
    #     (today - timedelta(weeks=w)).isocalendar()[:2]
    #     for w in range(weeks_back)
    # ]
    
    # 주차 리스트 생성 (이번 주 제외, 저번주 & 저저번주만)
    if weekday <= 2:
        # 월~수
        week_year_pairs = week_year_pairs = [
        (today - timedelta(weeks=2)).isocalendar()[:2],  # 저저번주
        (today - timedelta(weeks=1)).isocalendar()[:2],  # 저번주
        ]
    else:
        weeks_year_pairs = [
            (today - timedelta(weeks=1)).isocalendar()[:2],
            today.isocalendar()[:2]
        ]

    # 결과 저장용 딕셔너리 초기화
    merged_data_by_table = {table_name: {} for table_name in table_list}

    # 주차별로 데이터 로딩
    for year_val, week_val in week_year_pairs:
        for table_name in table_list:
            prefix = f'instagram-data/tables/{table_name}/year={year_val}/week={week_val}/'
            response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

            if 'Contents' not in response:
                print(f"[Info] No files found under prefix: {prefix}")
                continue

            target_files = [
                content['Key']
                for content in response['Contents']
                if content['Key'].endswith(target_filename)
            ]

            if not target_files:
                print(f"[Info] No {target_filename} found for {table_name} week={week_val}")
                continue

            for file_key in target_files:
                try:
                    obj = client.get_object(Bucket=bucket_name, Key=file_key)
                    df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
                    merged_data_by_table[table_name][week_val] = df
                    print(f"[Success] Loaded {file_key} for table {table_name}, week {week_val}")
                except Exception as e:
                    print(f"[Error] Failed to read {file_key} for {table_name}, week {week_val}: {e}")

    recent_weeks_data = {}
    for table_name, week_data in merged_data_by_table.items():
        sorted_weeks = sorted(week_data.keys())
        if len(sorted_weeks) >= 2:
            prev_week, current_week = sorted_weeks[-2], sorted_weeks[-1]
            # print(prev_week, current_week)
            recent_weeks_data[table_name] = {
                'prev_week': week_data[prev_week],
                'current_week': week_data[current_week]
            }
        else:
            print(f"[Warning] Not enough data for table {table_name} to determine prev/current weeks.")

    
    return recent_weeks_data


In [167]:
bucket_name = 'flexmatch-data'
table_list = [
    'RECENT_USER_INFO_MTR',
    'TIME_SERIES_PROFILE_INFO',
    'BY_USER_ID_MEDIA_DTL_INFO',
    'BY_DATE_MEDIA_AGG_INFO',
    'CONN_PROFILE_INSIGHT_DTL',
    'CONN_MEDIA_INSIGHT'
]

merged_data_by_table = load_weekly_instagram_data(bucket_name, table_list)


[Success] Loaded instagram-data/tables/RECENT_USER_INFO_MTR/year=2025/week=25/merged_data.parquet for table RECENT_USER_INFO_MTR, week 25
[Success] Loaded instagram-data/tables/TIME_SERIES_PROFILE_INFO/year=2025/week=25/merged_data.parquet for table TIME_SERIES_PROFILE_INFO, week 25
[Success] Loaded instagram-data/tables/BY_USER_ID_MEDIA_DTL_INFO/year=2025/week=25/merged_data.parquet for table BY_USER_ID_MEDIA_DTL_INFO, week 25
[Success] Loaded instagram-data/tables/BY_DATE_MEDIA_AGG_INFO/year=2025/week=25/merged_data.parquet for table BY_DATE_MEDIA_AGG_INFO, week 25
[Success] Loaded instagram-data/tables/CONN_PROFILE_INSIGHT_DTL/year=2025/week=25/merged_data.parquet for table CONN_PROFILE_INSIGHT_DTL, week 25
[Success] Loaded instagram-data/tables/CONN_MEDIA_INSIGHT/year=2025/week=25/merged_data.parquet for table CONN_MEDIA_INSIGHT, week 25
[Success] Loaded instagram-data/tables/RECENT_USER_INFO_MTR/year=2025/week=26/merged_data.parquet for table RECENT_USER_INFO_MTR, week 26
[Success

In [189]:
recent_user_info_mtr = merged_data_by_table['RECENT_USER_INFO_MTR']['prev_week']
time_series_profile_info = merged_data_by_table['TIME_SERIES_PROFILE_INFO']['prev_week']
by_user_id_media_dtl_info = merged_data_by_table['BY_USER_ID_MEDIA_DTL_INFO']['prev_week']
by_date_media_agg_info = merged_data_by_table['BY_DATE_MEDIA_AGG_INFO']['prev_week']

recent_user_info_mtr_2 = merged_data_by_table['RECENT_USER_INFO_MTR']['current_week']
time_series_profile_info_2 = merged_data_by_table['TIME_SERIES_PROFILE_INFO']['current_week']
by_user_id_media_dtl_info_2 = merged_data_by_table['BY_USER_ID_MEDIA_DTL_INFO']['current_week']
by_date_media_agg_info_2 = merged_data_by_table['BY_DATE_MEDIA_AGG_INFO']['current_week']

conn_profile_insight = merged_data_by_table['CONN_PROFILE_INSIGHT_DTL']['prev_week']
conn_profile_insight_2 = merged_data_by_table['CONN_PROFILE_INSIGHT_DTL']['current_week']

conn_media_insight = merged_data_by_table['CONN_MEDIA_INSIGHT']['prev_week']
conn_media_insight_2 = merged_data_by_table['CONN_MEDIA_INSIGHT']['current_week']

basic preprocessing

In [ ]:
# recent_user_info_mtr = recent_user_info_mtr.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# time_series_profile_info = time_series_profile_info.drop_duplicates(subset=['base_ymd', 'acnt_id'])
# by_user_id_media_dtl_info = by_user_id_media_dtl_info.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
# by_date_media_agg_info = by_date_media_agg_info.drop_duplicates(subset=['base_ymd', 'media_id'])

# recent_user_info_mtr_2 = recent_user_info_mtr_2.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# time_series_profile_info_2 = time_series_profile_info_2.drop_duplicates(subset=['base_ymd', 'acnt_id'])
# by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
# by_date_media_agg_info_2 = by_date_media_agg_info_2.drop_duplicates(subset=['base_ymd', 'media_id'])

# conn_profile_insight = conn_profile_insight.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# conn_media_insight = conn_media_insight.drop_duplicates(subset=['acnt_id', 'acnt_nm', 'media_id'])

# conn_profile_insight_2 = conn_profile_insight_2.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# conn_media_insight_2 = conn_media_insight_2.drop_duplicates(subset=['acnt_id', 'acnt_nm', 'media_id'])


가장 최근 데이터로 isin 해줘야 할 것 같음. 그래야 1주일 전, 후를 비교할 수 있기 때문임

In [210]:
# 각 acnt_id가 있는 전부 있는 경우만 가능, isin으로 제거
# 근데 전 주차에는 없는 유저가 생길수도 있음
# media 같은 경우에도 최근 25개만 일단 가져오고 있기 때문에 일주일 단위여도 없을수도 있음

# 이렇게 해야하는 이유는 timeseries 변수를 계산하려면 무조건 두 주차에 같은 아이디가 있어야 함
# 또한, user info랑 media info 같이 써야하는 경우에는 어짜피 한번 거르고 있을 듯
# -> 따라서, 굳이 필수적으로 하지는 않아도 될듯함

# unique_user = recent_user_info_mtr['acnt_id'].unique()
c_unique_user = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_conn_yn']=='Y']['acnt_id'].to_list()
# print(len(c_unique_user))

c_recent_user_info_mtr_2 = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_id'].isin(c_unique_user)]

c_time_series_profile_info = time_series_profile_info[time_series_profile_info['acnt_id'].isin(c_unique_user)]
c_time_series_profile_info_2 = time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(c_unique_user)]

# by_user_id_media_dtl_info = by_user_id_media_dtl_info[by_user_id_media_dtl_info['acnt_id'].isin(c_unique_user)]
c_by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2[by_user_id_media_dtl_info_2['acnt_id'].isin(c_unique_user)]

unique_media = c_by_user_id_media_dtl_info_2['media_id'].unique()
# connected_user 같은 경우에는 conn_media_insight 안에 있는 게 media_agg랑 동일하기 때문에 해당 부분을 쓰면 당장은 문제가 없음
# c_by_date_media_agg_info_2 = by_date_media_agg_info_2[by_date_media_agg_info_2['media_id'].isin(unique_media)]
conn_media_insight_2 = conn_media_insight_2[conn_media_insight_2['media_id'].isin(unique_media)]

In [171]:
def influencer_scale_type(row):
    count = row['follower_cnt']
    if count < 1000:
        return 'nano'
    elif 1000 <= count <= 10000:
        return 'micro'
    elif 10000 < count <= 100000:
        return 'mid'
    elif 100000 < count <= 500000:
        return 'macro'
    else:
        return 'mega'

# recent_user_info_mtr.loc[:, 'influencer_scale_type'] = recent_user_info_mtr.apply(influencer_scale_type, axis=1)
c_recent_user_info_mtr_2.loc[:,'influencer_scale_type'] = c_recent_user_info_mtr_2.apply(influencer_scale_type, axis=1)


In [140]:
# c_recent_user_info_mtr_2

In [211]:
# conn_media_insight_2[conn_media_insight_2['acnt_id']=='17841450980480576'].head()

In [212]:
# # by_user_id_media_dtl_info_2[by_user_id_media_dtl_info_2['acnt_id']=='17841450980480576'] # 17850258761802532
# by_date_media_agg_info_2[by_date_media_agg_info_2['media_id']=='17850258761802532']

In [213]:
# conn_profile_insight_2[conn_profile_insight_2['acnt_id']=='17841450980480576']

create merged file

In [219]:
def create_merged_df(user_info_df, timeseries_df, timeseries_df_2, media_info_df, media_insight): # media_agg, profile_insight X
    media_engagement_merged_df = pd.merge(media_info_df, media_insight, on='media_id', how='outer')
    # print(len(media_engagement_merged_df['acnt_id'].unique()))

    # 단 한개의 게시물이라도 like가 비공개인 influencer 제거
    # by_user_na_like_count = media_engagement_merged_df[media_engagement_merged_df['like_cnt'].isna()].groupby(['acnt_id'])['media_id'].count()
    # na_like_user = by_user_na_like_count[by_user_na_like_count > 0].index
    # # print(len(na_like_user))
    # media_engagement_merged_df = media_engagement_merged_df[~media_engagement_merged_df['acnt_id'].isin(na_like_user)].reset_index()

    user_list = media_engagement_merged_df['acnt_id'].unique()
    # print(len(user_list))
    media_list = media_engagement_merged_df['media_id'].unique()

    # merge하면서 제거된 리스트가 있기 때문에, 해당 부분 다시 삭제 후에 새로운 merge 파일 생성
    user_info = user_info_df[user_info_df['acnt_id'].isin(user_list)]
    timeseries = timeseries_df[timeseries_df['acnt_id'].isin(user_list)]
    timeseries_2 = timeseries_df_2[timeseries_df_2['acnt_id'].isin(user_list)]
    media_info = media_info_df[media_info_df['acnt_id'].isin(user_list)]

    media_insight_info = media_insight[media_insight['media_id'].isin(media_list)]

    all_merged_df_a = pd.merge(user_info, timeseries, on='acnt_id')
    all_merged_df_b = pd.merge(all_merged_df_a, media_info, on='acnt_id')
    all_merged_df = pd.merge(all_merged_df_b, media_insight_info, on='media_id')
    
    media_engagement_profile_merged_df = pd.merge(media_engagement_merged_df, user_info_df, on='acnt_id')
    time_series_merged_df = pd.merge(timeseries, timeseries_df_2, on='acnt_id')

    return user_info, timeseries, timeseries_2, media_info, media_insight_info, all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df

In [220]:
user_info, timeseries, timeseries_2, media_info, media_agg, all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df = create_merged_df(
                                                                                                                        c_recent_user_info_mtr_2,
                                                                                                                        c_time_series_profile_info,
                                                                                                                        c_time_series_profile_info_2,
                                                                                                                        c_by_user_id_media_dtl_info_2,
                                                                                                                        conn_media_insight_2)

KeyError: 'acnt_id'

In [218]:
media_engagement_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   acnt_id             337 non-null    object        
 1   media_id            337 non-null    object        
 2   media_type_nm       337 non-null    object        
 3   reels_feed_type_nm  337 non-null    object        
 4   media_url_addr      337 non-null    object        
 5   media_unq_url_addr  337 non-null    object        
 6   tmnl_url_addr       39 non-null     object        
 7   reg_dt              337 non-null    object        
 8   media_cn            274 non-null    object        
 9   acnt_conn_yn        337 non-null    object        
 10  feed_share_yn       31 non-null     object        
 11  cmnt_actvtn_yn      337 non-null    object        
 12  base_ymd            0 non-null      object        
 13  cmnt_cnt            0 non-null      float64       

In [216]:
media_engagement_merged_df[media_engagement_merged_df['acnt_id']=='17841450980480576']

,acnt_id,media_id,media_type_nm,reels_feed_type_nm,media_url_addr,media_unq_url_addr,tmnl_url_addr,reg_dt,media_cn,acnt_conn_yn,feed_share_yn,cmnt_actvtn_yn,base_ymd,cmnt_cnt,like_cnt,created_dt
5,17841450980480576,17845632434762696,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/CcFHLyjJym7/,None,2022-04-08T06:23:29+0000,#34\n🔵 🧊🧊🧊🧊🧊\n클라임 어스 첫 방문\n시설 훌륭하네\n남색 어렵다\n클라...,Y,None,true,NaN,NaN,NaN,NaT
15,17841450980480576,17850258761802532,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CgqtlLjJ9Z9/,None,2022-07-31T06:56:29+0000,#64회차 \n⚫️⚫️❌️🔴🔴🔴🔴🔴🔴\n손상원 클라이밍짐 강남점 오픈런!\n처음 만...,Y,None,true,NaN,NaN,NaN,NaT
21,17841450980480576,17856317744695367,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/CZoOOpFpPUP/,None,2022-02-06T08:04:17+0000,.\n🧊🧊🧊🧊🧊 좋다 좀 늘긴했어ㅎㅎ\n🟢 다이노라서 재밌다\n🔵 이거 진짜 깨고 ...,Y,None,true,NaN,NaN,NaN,NaT
23,17841450980480576,17857189607706657,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CdiBqPup5KM/,None,2022-05-14T08:24:35+0000,#45회차\n🔵🔵🔵🔵🔵🔵\n웨어하우스짐 첫 방문\n깔끔하고 문제도 재밌고 사장님도 ...,Y,None,true,NaN,NaN,NaN,NaT
28,17841450980480576,17859119504684701,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/Ca3xraepfP3/,None,2022-03-09T05:34:03+0000,.\n오랜만에 양재 더클라임\n🔵x2 다이노\n참교육만 엄청 당했다\n우연히 만난\...,Y,None,true,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,17841450980480576,18229115269104698,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cb4ZC0qJaBM/,None,2022-04-03T07:49:25+0000,#33\n🟣 물보라든 꿀보라든 보라!!\n❌️ 기세를 몰아 다른 보라 도전했지만 탑...,Y,None,true,NaN,NaN,NaN,NaT
314,17841450980480576,18237946321160191,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/CqhrVJVy91h/,None,2023-04-02T07:57:35+0000,04/02 미금 더볼더\n❌️🧊🧊🧊🧊\n클라이밍 1일차~\n4달만에 @_p.ark ...,Y,None,true,NaN,NaN,NaN,NaT
315,17841450980480576,18240952645111314,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/CeqHzhOJBAv/,None,2022-06-11T08:23:36+0000,#53회차\n🟢🟢❌️🟢🟢🟢\n비블럭 강남은 참 재밌다\n오버행 하나 못깬거 너무 아...,Y,None,true,NaN,NaN,NaN,NaT
319,17841450980480576,18295822177054619,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cdadlu6J4Mc/,None,2022-05-11T09:54:43+0000,#44회차\nDD❌️EE\n내 인생 첫 클라이밍했던 곳\n앞으로 내 주암장이 될 곳...,Y,None,true,NaN,NaN,NaN,NaT


크리에이터 활동성

- 실제 게시물 업로드 날짜를 기준으로 계산할 수 있을 것 같음
- 최근 25개의 게시물의 평균 게시 간격을 계산
- 최근 25개의 게시물을 며칠에 걸쳐서 업로드 했는지에 대한 값을 계산

In [99]:
media_dtl_2_copy = by_user_id_media_dtl_info_2.copy()
media_dtl_2_copy = media_dtl_2_copy.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
media_dtl_2_copy['reg_dt'] = pd.to_datetime(media_dtl_2_copy['reg_dt'])
media_dtl_2_copy = media_dtl_2_copy.sort_values(['acnt_id', 'reg_dt'])

# 게시물 간격 계산
media_dtl_2_copy['prev_reg_dt'] = media_dtl_2_copy.groupby('acnt_id')['reg_dt'].shift(1)
media_dtl_2_copy['gap_days'] = (media_dtl_2_copy['reg_dt'] - media_dtl_2_copy['prev_reg_dt']).dt.days
# media_dtl_2_copy

# gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
activity_df = media_dtl_2_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

# 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
activity_df['activity_score'] = 100 / activity_df['avg_upload_interval']
activity_df

,acnt_id,avg_upload_interval,activity_score
0,17841400002751973,0.333333,300.000000
1,17841400006499942,3.166667,31.578947
2,17841400011995621,2.291667,43.636364
3,17841400016295781,0.541667,184.615385
4,17841400025364370,26.416667,3.785489
...,...,...,...
716,17841473332885823,2.750000,36.363636
717,17841473347851502,2.208333,45.283019
718,17841473857435791,0.500000,200.000000
719,17841474152637635,3.500000,28.571429


In [76]:
def calculate_activity_score(recent_media_dtl_df): # 두 개의 테이블 중 가장 최근
    media_dtl_copy = recent_media_dtl_df.copy()
    media_dtl_copy = media_dtl_copy.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
    media_dtl_copy['reg_dt'] = pd.to_datetime(media_dtl_copy['reg_dt'])
    media_dtl_copy = media_dtl_copy.sort_values(['acnt_id', 'reg_dt'])

    # 게시물 간격 계산
    media_dtl_copy['prev_reg_dt'] = media_dtl_copy.groupby('acnt_id')['reg_dt'].shift(1)
    media_dtl_copy['gap_days'] = (media_dtl_copy['reg_dt'] - media_dtl_copy['prev_reg_dt']).dt.days

    # gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
    activity_df = media_dtl_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
    activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

    # 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
    # 업로드 간격이 너무 짧은 유저의 경우 inf로 계산되는 것을 방지하기 위해서 scaling 진행
    activity_df['avg_upload_interval'] = activity_df['avg_upload_interval'].replace(0, 0.1)
    activity_df['activity_score'] = 100 / activity_df['avg_upload_interval']
    
    return activity_df

In [78]:
activity_df = calculate_activity_score(media_info)

In [79]:
activity_df

,acnt_id,avg_upload_interval,activity_score
0,17841400337192059,1044.000000,0.095785
1,17841400360358101,120.000000,0.833333
2,17841400624415491,21.545455,4.641350
3,17841409045873013,292.000000,0.342466
4,17841449549923448,3.070707,32.565789
5,17841450980480576,12.070707,8.284519
6,17841470887323678,0.100000,1000.000000


In [80]:
def check_inf(df):
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns

    mask_inf = np.isinf(df[float_cols]).any(axis=1)
    mask_neginf = np.isneginf(df[float_cols]).any(axis=1)

    invalid_rows = df[mask_inf | mask_neginf]

    print(f"⚠️ inf / -inf 포함 행 개수: {len(invalid_rows)}개")
    display(invalid_rows)

In [81]:
check_inf(activity_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,avg_upload_interval,activity_score


트렌드지수 - 팔로워 순증감률 (follower_growth_rate)

In [126]:
# time_series_profile_info_2['acnt_id'] = time_series_profile_info_2['acnt_id'].astype(object)

# influencer_list = time_series_profile_info['acnt_id'].unique()
# time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(influencer_list)]
# time_series_merged_df = pd.merge(time_series_profile_info, time_series_profile_info_2, on='acnt_id')

# time_series_merged_df['trend_score'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
# time_series_merged_df[['acnt_id', 'trend_score']]

In [82]:
def calculate_follower_growth_rate(time_series_df, recent_time_series_df):
    time_series_df.loc[:, 'acnt_id'] = time_series_df['acnt_id'].astype(object)
    recent_time_series_df.loc[:, 'acnt_id'] = recent_time_series_df['acnt_id'].astype(object)

    influencer_list = time_series_df['acnt_id'].unique()
    recent_time_series_df = recent_time_series_df[recent_time_series_df['acnt_id'].isin(influencer_list)]
    time_series_merged_df = pd.merge(time_series_df, recent_time_series_df, on='acnt_id')

    time_series_merged_df['trend_score'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
    growth_rate_df = time_series_merged_df[['acnt_id', 'trend_score']]

    return growth_rate_df

In [83]:
growth_rate_df = calculate_follower_growth_rate(timeseries, timeseries_2)

In [84]:
growth_rate_df

,acnt_id,trend_score
0,17841449549923448,-0.184290
1,17841400624415491,0.000000
2,17841450980480576,0.000000
3,17841400337192059,0.000000
4,17841400360358101,2.419355
5,17841409045873013,0.952381
6,17841470887323678,1.250000


##### 팔로워 참여도

- estimated_total_engagement = avg_engagement_per_post * total_post_count
- avg_engagement_per_post = like+comments / 25 
  -> 25개가 아니라 비즈니스 연동 계정 같은 경우 max 100임. 이거 어케 나눠줘야 할지? (id별 goupby해서 개수를 알아야 할 듯)
- total_post_count -> user_data

In [135]:
def calculate_follower_engagement(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df[['acnt_id', 'media_id', 'follower_cnt', 'follow_cnt', 'like_cnt', 'cmnt_cnt', 'media_cnt']]
    
    engaged_df = media_engagement_profile_merged_df_copy.groupby(['acnt_id']).agg({
        'like_cnt' : 'sum',
        'cmnt_cnt' : 'sum',
        'media_cnt': 'first',
        'follower_cnt' : 'first',
        'follow_cnt' : 'first'
    }).reset_index()

    engaged_df['avg_engagement_per_post'] = ((engaged_df['like_cnt'] + engaged_df['cmnt_cnt']) / 25)
    engaged_df['estimated_total_engagement'] = engaged_df['avg_engagement_per_post'] * engaged_df['media_cnt']
    engaged_df['follower_total_engagement'] = (engaged_df['estimated_total_engagement'] / engaged_df['follower_cnt']) * 100
    
    follower_engagment_df = engaged_df

    return follower_engagment_df

In [136]:
follower_engagment_df = calculate_follower_engagement(media_engagement_profile_merged_df)

In [137]:
check_inf(follower_engagment_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,like_cnt,cmnt_cnt,media_cnt,follower_cnt,follow_cnt,avg_engagement_per_post,estimated_total_engagement,follower_total_engagement


##### 팔로워 충성도

- user_info or time_series 데이터가 필요
- 트렌드 지수는 time_series 데이터를 가지고 단순히 빼기를 진행했었음

In [ ]:
# time_series_merged_df_copy = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'follower_cnt_y']]

# time_series_merged_df_copy['follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])

# def estimate_new_follower(row):
#     if row['follower_change'] < 0:
#         return 0
#     else:
#         return row['follower_change']

# time_series_merged_df_copy['new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)

# # 충성도 계산
# # time_series_merged_df_copy = time_series_merged_time_series_merged_df_copy_copy.copy()
# time_series_merged_df_copy['unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y']
# time_series_merged_df_copy['follower_retention_rate'] = ((time_series_merged_df_copy['follower_cnt_x'] - time_series_merged_df_copy['unfollowed']) / time_series_merged_df_copy['follower_cnt_x']) * 100
# time_series_merged_df_copy['follower_retention_rate'] = time_series_merged_df_copy['follower_retention_rate'].round(2)

# time_series_merged_df_copy

In [ ]:
# def estimate_loyalty(row):
#     if row['follower_change'] < 0:
#         return ((row['follower_cnt_x'] + row['follower_change']) / row['follower_cnt_x']) * 100
#     else:
#         return 100.0

# time_series_merged_df_copy['follower_loyalty'] = time_series_merged_df_copy.apply(estimate_loyalty, axis=1)
# time_series_merged_df_copy['follower_loyalty'] = time_series_merged_df_copy['follower_loyalty'].round(2)


In [ ]:
def calculate_follower_loyalty(time_series_merged_df):
    time_series_merged_df_copy = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'follower_cnt_y']].copy()

    time_series_merged_df_copy.loc[:, 'follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])

    def estimate_new_follower(row):
        if row['follower_change'] < 0:
            return 0
        else:
            return row['follower_change']

    time_series_merged_df_copy.loc[:, 'new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)
    time_series_merged_df_copy.loc[:, 'unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y']
    time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = ((time_series_merged_df_copy['follower_cnt_x'] - time_series_merged_df_copy['unfollowed']) / time_series_merged_df_copy['follower_cnt_x']) * 100
    time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = time_series_merged_df_copy['follower_retention_rate'].round(2)

    follower_loyalty_df = time_series_merged_df_copy

    return follower_loyalty_df

In [143]:
follower_loyalty_df = calculate_follower_loyalty(time_series_merged_df)
follower_loyalty_df

,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate
0,17841472095454621,2348,2424,76,76,0,100.00
1,17841459031207937,2119,2119,0,0,0,100.00
2,17841458105192949,1269,1271,2,2,0,100.00
3,17841400305959659,12337,12319,-18,0,18,99.85
4,17841418684451303,21360,20696,-664,0,664,96.89
...,...,...,...,...,...,...,...
357,17841401900622950,24985,24944,-41,0,41,99.84
358,17841402240386451,1437,1436,-1,0,1,99.93
359,17841411620779046,1069,1065,-4,0,4,99.63
360,17841400875316568,1267,1266,-1,0,1,99.92


In [142]:
check_inf(follower_loyalty_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate


콘텐츠 효율성 평가

- 콘텐츠 1개당 팔로워 대비 반응량(like+comments)
- media_agg & media_dtl & acnt_id가 필요 -> 전부 병합한 게 media_engagement_merged_df

In [132]:
# media_engagement_merged_df_3 = media_engagement_profile_merged_df.copy()
# # media_engagment_merged_df_2 == media_engagement_profile_merged_df 같은 df임
# # media_engagement_profile_merged_df_copy = media_engagement_merged_df_2.copy()


# media_engagement_merged_df_3['post_efficiency'] = ((media_engagement_merged_df_3['like_cnt'] + media_engagement_merged_df_3['cmnt_cnt']) / media_engagement_merged_df_3['follower_cnt']) * 100
# post_efficiency_df = media_engagement_merged_df_3.groupby('acnt_id')['post_efficiency'].mean().reset_index()
# post_efficiency_df.rename(columns={'post_efficiency': 'avg_post_efficiency'}, inplace=True)
# post_efficiency_df

In [144]:
def calculate_post_efficiency_df(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df.copy()

    media_engagement_profile_merged_df_copy['post_efficiency'] = ((media_engagement_profile_merged_df_copy['like_cnt'] + media_engagement_profile_merged_df_copy['cmnt_cnt']) / media_engagement_profile_merged_df_copy['follower_cnt']) * 100
    post_efficiency_df = media_engagement_profile_merged_df_copy.groupby('acnt_id')['post_efficiency'].mean().reset_index()
    post_efficiency_df.rename(columns={'post_efficiency': 'avg_post_efficiency'}, inplace=True)

    return post_efficiency_df

In [145]:
post_efficiency_df = calculate_post_efficiency_df(media_engagement_profile_merged_df)

In [146]:
check_inf(post_efficiency_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,avg_post_efficiency


##### not_connected_influencer_flexmatch_score

In [54]:
# # 크리에이터 활동성
# creator_activity_score = activity_df[['acnt_id', 'activity_score']]
# # 트렌드지수
# creator_trend_score = growth_rate_df[['acnt_id', 'trend_score']]
# # 팔로워 참여도
# follower_engagement = follower_engagment_df[['acnt_id', 'follower_total_engagement']]
# # 팔로워 충성도
# follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
# # 콘텐츠 효율성
# post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]

# # data_list
# df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency]

# from functools import reduce

# flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)
# user_info_nm = recent_user_info_mtr[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
# # user_info_nm
# flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
# flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'activity_score', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency']]

# flexmatch_score.rename(columns={
#     'avg_upload_interval' : '크리에이터 활동성',
#     'trend_score' : '트렌드지수(팔로워순증가량)', 
#     'follower_total_engagement' : '콘텐츠 참여도' ,
#     'follower_retention_rate' : '팔로워 충성도',
#     'avg_post_efficiency' : '콘텐츠 효율성',
# }, inplace=True)

# flexmatch_score_2 = flexmatch_score.copy()
# flexmatch_score_2.dropna(inplace=True)
# flexmatch_score_2

In [216]:
def not_connected_user_flexmatch_score(activity_df, growth_rate_df, follower_engagement_df, follower_loyalty_df, post_efficiency_df):
    # 크리에이터 활동성
    creator_activity_score = activity_df[['acnt_id', 'activity_score']]
    # 트렌드지수
    creator_trend_score = growth_rate_df[['acnt_id', 'trend_score']]
    # 팔로워 참여도
    follower_engagement = follower_engagement_df[['acnt_id', 'follower_total_engagement']]
    # 팔로워 충성도
    follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
    # 콘텐츠 효율성
    post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]

    # data_list
    df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency]

    from functools import reduce

    flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)
    user_info_nm = recent_user_info_mtr[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
    flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
    flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'influencer_scale_type', 'activity_score', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency']]

    # flexmatch_score.rename(columns={
    #     'activity_score' : '크리에이터 활동성',
    #     'trend_score' : '팔로워순증감률', 
    #     'follower_total_engagement' : '콘텐츠 참여도' ,
    #     'follower_retention_rate' : '팔로워 충성도',
    #     'avg_post_efficiency' : '콘텐츠 효율성',
    # }, inplace=True)

    not_connected_flexmatch_score_table = flexmatch_score.copy()
    not_connected_flexmatch_score_table.dropna(inplace=True)
    
    return not_connected_flexmatch_score_table

In [217]:
not_connected_flexmatch_score_table = not_connected_user_flexmatch_score(activity_df, growth_rate_df, follower_engagment_df, follower_loyalty_df, post_efficiency_df)
not_connected_flexmatch_score_table

,acnt_id,acnt_nm,influencer_scale_type,activity_score,trend_score,follower_total_engagement,follower_retention_rate,avg_post_efficiency
0,17841400016295781,jihyolimm,mid,184.615385,0.000000,603.536863,100.00,1.221735
1,17841400038455668,kos9803,mid,11.940299,-0.020737,290.415000,99.98,0.335352
2,17841400064417438,jjosworlds,mid,141.176471,-0.404967,2484.457250,99.60,1.274734
3,17841400070132367,gnuoyeatt,nano,12.371134,-0.110375,1464.088398,99.89,13.812155
4,17841400079187566,jjjjeong_2,micro,16.666667,-0.110041,466.945745,99.89,0.782154
...,...,...,...,...,...,...,...,...
355,17841472245916641,bella_haus_,micro,20.183486,39.187956,345.354310,100.00,16.321092
356,17841473211163104,mom._.room,mid,200.000000,17.509025,595.526882,100.00,12.946237
357,17841473347851502,hyepick_place,micro,45.283019,-0.882292,109.016651,99.12,3.633888
358,17841473857435791,sesebonbon,micro,200.000000,7.234432,347.076923,100.00,8.676923


In [218]:
not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['acnt_id']=='17841400111322589']

,acnt_id,acnt_nm,influencer_scale_type,activity_score,trend_score,follower_total_engagement,follower_retention_rate,avg_post_efficiency
6,17841400111322589,seon4123,nano,266.666667,-0.206186,1053.487603,99.79,1.024793


In [219]:
not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['acnt_id']=='17841400070132367']

,acnt_id,acnt_nm,influencer_scale_type,activity_score,trend_score,follower_total_engagement,follower_retention_rate,avg_post_efficiency
3,17841400070132367,gnuoyeatt,nano,12.371134,-0.110375,1464.088398,99.89,13.812155


In [220]:
not_connected_flexmatch_score_table['influencer_scale_type'].value_counts()

influencer_scale_type
micro    146
mid      105
nano      93
macro     15
mega       1
Name: count, dtype: int64

In [221]:
nano = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='nano']
micro = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='micro']
mid = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='mid']
macro = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='macro']
mega = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='mega']

In [222]:
# influencer_scale_df_list = [nano, micro, mid, macro, mega]


# for df in influencer_scale_df_list :
#     columns = df.select_dtypes(include='float64').columns
#     cleaned = df[columns].replace([np.inf, -np.inf], np.nan).dropna()
#     minmax = MinMaxScaler(feature_range=(0, 5))
#     flexmatch_score_norm = minmax.fit_transform(cleaned[columns])

#     flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=cleaned.index)
#     flexmatch_score_norm_df['acnt_id'] = df['acnt_id']
#     flexmatch_score_norm_df['acnt_nm'] = df['acnt_nm']

#     flexmatch_score_norm_df['크리에이터 활동성'] = 5 - flexmatch_score_norm_df['크리에이터 활동성']

In [223]:
influencer_scale_df_list = [nano, micro, mid, macro, mega]
normalized_df_list = []

# 높을수록 안 좋은 지표
reverse_columns = []  

for df in influencer_scale_df_list:
    cleaned = df.copy()

    # 무한대 및 NaN 제거
    float_cols = cleaned.select_dtypes(include='float64').columns
    cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
    cleaned = cleaned.dropna(subset=float_cols)

    if cleaned.empty:
        continue

    # 개별 컬럼 정규화 (0~5 스케일)
    norm_df = pd.DataFrame(index=cleaned.index)
    for col in float_cols:
        scaler = MinMaxScaler(feature_range=(0, 5))
        norm_col = scaler.fit_transform(cleaned[[col]])
        # norm_df[col] = 5 - norm_col.ravel() if col in reverse_columns else norm_col.ravel()
        norm_df[col] = norm_col.ravel() if col in reverse_columns else norm_col.ravel()

    # ID 및 이름 붙이기
    norm_df['acnt_id'] = cleaned['acnt_id'].values
    norm_df['acnt_nm'] = cleaned['acnt_nm'].values

    normalized_df_list.append(norm_df)


In [224]:
influencer_scale_names = ['nano', 'micro', 'mid', 'macro', 'mega']
influencer_scale_df_list = [nano, micro, mid, macro, mega]
normalized_df_dict = {}
# normalized_df_list = []

reverse_columns = []

for name, df in zip(influencer_scale_names, influencer_scale_df_list):
    cleaned = df.copy()

    # 무한대 및 NaN 제거
    float_cols = cleaned.select_dtypes(include='float64').columns
    cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
    cleaned = cleaned.dropna(subset=float_cols)

    if cleaned.empty:
        continue

    norm_df = pd.DataFrame(index=cleaned.index)
    for col in float_cols:
        scaler = MinMaxScaler(feature_range=(0, 5))
        norm_col = scaler.fit_transform(cleaned[[col]])
        # norm_df[col] = 5 - norm_col.ravel() if col in reverse_columns else norm_col.ravel()
        norm_df[col] = norm_col.ravel() if col in reverse_columns else norm_col.ravel()

    # ID 및 이름 붙이기
    norm_df['acnt_id'] = cleaned['acnt_id'].values
    norm_df['acnt_nm'] = cleaned['acnt_nm'].values
    norm_df['influencer_scale_type'] = name

    # 딕셔너리에 저장
    normalized_df_dict[name] = norm_df

normalized_all_df = pd.concat(normalized_df_dict, ignore_index=True)
normalized_all_dic = normalized_all_df.to_dict(orient='index')


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

def normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list, reverse_columns=None, feature_range=(0, 5)):

    if reverse_columns is None:
        reverse_columns = []

    normalized_df_dict = {}

    for name, df in zip(influencer_scale_names, influencer_scale_df_list):
        cleaned = df.copy()

        # 무한대 및 NaN 제거
        float_cols = cleaned.select_dtypes(include='float64').columns
        cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
        cleaned = cleaned.dropna(subset=float_cols)

        if cleaned.empty:
            continue

        norm_df = pd.DataFrame(index=cleaned.index)
        for col in float_cols:
            scaler = MinMaxScaler(feature_range=feature_range)
            norm_col = scaler.fit_transform(cleaned[[col]])
            if col in reverse_columns:
                norm_df[col] = feature_range[1] - norm_col.ravel()
            else:
                norm_df[col] = norm_col.ravel()

        # ID 및 이름, 스케일 타입 추가
        norm_df['acnt_id'] = cleaned['acnt_id'].values
        norm_df['acnt_nm'] = cleaned['acnt_nm'].values
        norm_df['influencer_scale_type'] = name

        normalized_df_dict[name] = norm_df

    normalized_all_df = pd.concat(normalized_df_dict.values(), ignore_index=True)
    normalized_all_dic = normalized_all_df.to_dict(orient='index')

    return normalized_all_df, normalized_all_dic


In [255]:
influencer_scale_names=['nano', 'micro', 'mid', 'macro', 'mega']
influencer_scale_df_list=[nano, micro, mid, macro, mega]

normalized_df, normalized_dic = normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list)


In [256]:
normalized_all_dic

{0: {'activity_score': 0.18618008163528707,
  'trend_score': 1.2384506112322682,
  'follower_total_engagement': 0.9006925508419993,
  'follower_retention_rate': 4.84593837535013,
  'avg_post_efficiency': 1.4776093002849746,
  'acnt_id': '17841400070132367',
  'acnt_nm': 'gnuoyeatt',
  'influencer_scale_type': 'nano'},
 1: {'activity_score': 4.442126007071234,
  'trend_score': 1.2041673155386674,
  'follower_total_engagement': 0.6477789738723821,
  'follower_retention_rate': 4.70588235294116,
  'avg_post_efficiency': 0.07223635090475532,
  'acnt_id': '17841400111322589',
  'acnt_nm': 'seon4123',
  'influencer_scale_type': 'nano'},
 2: {'activity_score': 0.7667208771875991,
  'trend_score': 1.338593886015669,
  'follower_total_engagement': 3.2795727597981803,
  'follower_retention_rate': 5.0,
  'avg_post_efficiency': 0.3240928471358846,
  'acnt_id': '17841400204233993',
  'acnt_nm': 'j_solha',
  'influencer_scale_type': 'nano'},
 3: {'activity_score': 0.20735524256651017,
  'trend_score'

##### 크리에이터 광고 효율성 

In [ ]:
revenue_dic = {
    'acnt_nm' : ['s_h_j_', 'siwolbubu_hyun', 'bong_camper83', 'binwoos', 'seojinii_', 'tingkerhee'],
    'sell_type' : ['flexmatch', 'other', 'flexmatch', 'flexmatch', 'other', 'other'],
    'total_revenue' : [6906000, 10937105, 7233100, 8759000, 7939664, 38449720],
    'total_order_cnt' : [84, 132, 122, 88, 89, 471]
    }

revenue_df = pd.DataFrame(revenue_dic)
revenue_merged_df = pd.merge(media_engagement_merged_df, revenue_df, on='acnt_nm', how='left')
revenue_merged_df = revenue_merged_df[['acnt_id', 'acnt_nm', 'follower_cnt', 'follow_cnt', 'media_cnt', 'sell_type', 'total_revenue', 'total_order_cnt', 'media_id', 'like_cnt', 'cmnt_cnt']]
revenue_merged_df

revenue_merged_df['post_efficiency'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / revenue_merged_df['follower_cnt']) * 100
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'post_efficiency' : 'mean',
    'total_order_cnt' : 'first'
}).dropna()

revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['post_efficiency']) 
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

revenue_merged_df['engagement_per_post'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / 25)
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'engagement_per_post' : 'mean',
    'total_revenue' : 'first',
    'total_order_cnt' : 'first'
}).dropna()

# 팔로워를 생각하지 않고, 25개의 콘텐츠 단위당 반응 비율을 계산
revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['engagement_per_post'] * 25) * 100
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

,acnt_nm,sell_type,total_revenue,total_order_cnt
0,s_h_j_,flexmatch,6906000,84
1,siwolbubu_hyun,other,10937105,132
2,bong_camper83,flexmatch,7233100,122
3,binwoos,flexmatch,8759000,88
4,seojinii_,other,7939664,89
5,tingkerhee,other,38449720,471


콘텐츠 효율성이 아닌 팔로워 참여도를 기준으로 계산했을 때

In [ ]:
engaged_df_2 = engaged_df.copy()
revenue_merged_df_2 = revenue_merged_df.groupby(['acnt_id', 'acnt_nm'])[['total_revenue', 'total_order_cnt']].first().dropna().reset_index()
revenue_df_total_2 = pd.merge(engaged_df_2, revenue_merged_df_2, on='acnt_id')

revenue_df_total_2['advertisement_efficiency'] = revenue_df_total_2['total_order_cnt'] / revenue_df_total_2['follower_total_engagement']
revenue_df_total_2[['acnt_id', 'acnt_nm', 'total_revenue', 'total_order_cnt', 'follower_total_engagement', 'advertisement_efficiency']].sort_values(by='advertisement_efficiency', ascending=False)

table merged

In [583]:
# 크리에이터 활동성
creator_activity_score = activity_df[['acnt_id', 'avg_upload_interval']]

In [584]:
# 트렌드지수
creator_trend_score = time_series_merged_df[['acnt_id', 'trend_score']]

In [585]:
# 팔로워 참여도
follower_engagement = engaged_df[['acnt_id', 'follower_total_engagement']]

In [586]:
# 팔로워 충성도
follower_loyalty = time_series_merged_df_copy[['acnt_id', 'follower_retention_rate']]

In [587]:
# 콘텐츠 효율성
post_efficiency = post_efficiency_df.copy()

In [595]:
# 광고효율성
revenue_df_total = revenue_df_total.reset_index()
advertisement_efficiency = revenue_df_total[['acnt_id', 'advertisement_efficiency']]

In [596]:
df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency, advertisement_efficiency]

In [597]:
from functools import reduce

flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)

In [598]:
user_info_nm = user_info_2[['acnt_id', 'acnt_nm']]
flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')

In [604]:
flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'avg_upload_interval', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency', 'advertisement_efficiency']]
flexmatch_score.rename(columns={
    'avg_upload_interval' : '크리에이터 활동성',
    'trend_score' : '트렌드지수(팔로워순증가량)', 
    'follower_total_engagement' : '콘텐츠 참여도' ,
    'follower_retention_rate' : '팔로워 충성도',
    'avg_post_efficiency' : '콘텐츠 효율성',
    'advertisement_efficiency' : '광고 효율성'
}, inplace=True)

In [605]:
flexmatch_score_2 = flexmatch_score.copy()
flexmatch_score_2.dropna(inplace=True)

In [606]:
flexmatch_score_2

,acnt_id,acnt_nm,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성
0,17841400361359004,s_h_j_,1.166667,0.057866,110.188904,100.00,0.045178,1859.316062
1,17841400561503844,binwoos,0.500000,-0.014956,626.642600,99.99,0.169730,518.470975
2,17841400591698216,tingkerhee,2.416667,-0.018509,2065.579146,99.98,0.483403,974.343202
4,17841401506106699,siwolbubu_hyun,0.958333,-0.075045,2094.996055,99.92,0.556736,237.096389
6,17841402936102997,seojinii_,2.875000,-0.002312,166.809026,100.00,0.240359,370.279723
10,17841453615191128,bong_camper83,1.500000,0.281793,2526.578680,100.00,2.506526,48.672935


In [608]:
# from sklearn.preprocessing import MinMaxScaler

# columns = flexmatch_score_2.select_dtypes(include='float64').columns
# minmax = MinMaxScaler(feature_range=(0, 5))
# flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

# flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
# flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
# flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

# flexmatch_score_norm_df['avg_upload_interval'] = 5 - flexmatch_score_norm_df['avg_upload_interval']
# flexmatch_score_norm_df


In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns = flexmatch_score_2.select_dtypes(include='float64').columns
minmax = MinMaxScaler(feature_range=(0, 5))
flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

flexmatch_score_norm_df['크리에이터 활동성'] = 5 - flexmatch_score_norm_df['크리에이터 활동성']
flexmatch_score_norm_df

# 지금 이거 정규화 할 때는 그냥 이사람들을 전부 다 합쳐서 함께 정규화를 진행해서 그런데, 원래라면은 팔로워를 기준으로 뭐 메가 인플루언서, 마이크로 인플루언서 이런식으로 기준을 나눠서 정규화를 해야 조금 더 정확한 값이 될 것 같음.


,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성,acnt_id,acnt_nm
0,3.596491,1.862339,0.000000,5.000,0.000000,5.000000,17841400361359004,s_h_j_
1,5.000000,0.841960,1.068647,4.375,0.253016,1.297324,17841400561503844,binwoos
2,0.964912,0.792182,4.046099,3.750,0.890213,2.556192,17841400591698216,tingkerhee
4,4.035088,0.000000,4.106968,0.000,1.039182,0.520322,17841401506106699,siwolbubu_hyun
6,0.000000,1.019133,0.117159,5.000,0.396492,0.888101,17841402936102997,seojinii_
10,2.894737,5.000000,5.000000,5.000,5.000000,0.000000,17841453615191128,bong_camper83


In [ ]:
# flexmatch_score_norm_df.to_csv("flexmatch_score_test.csv")